In [1]:
!pip install datasets

In [2]:
!pip install transformers

In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.9 MB/s eta 0:00:00:00:01


In [8]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9c60e7b489401bd0142720698f1be1eea1e9d0db746e77f30c5d09c3699be0e0
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
# Import necessary libraries
import os
from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import torch.nn.functional as F
import re

from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [10]:
# Load dataset
ds = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 112165
    })
})

In [11]:
# # Shuffle and select a subset of the data
# full_dataset = ds['train'].shuffle(seed=42).select(range(11000))


# Select the first 15000 rows of the dataset
full_dataset = ds['train'].select(range(15000))

full_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 15000
})

In [12]:
# Split the data
train_testvalid = full_dataset.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

train_dataset = train_testvalid['train']
val_dataset = test_valid['train']
test_dataset = test_valid['test']

In [13]:
# Extract unique outputs to use as labels
unique_labels = list(set(train_dataset['output'] + val_dataset['output'] + test_dataset['output']))

# Create a label mapping
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}


# --- semantic search --- #
# Load a pre-trained sentence transformer model
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all possible responses
response_embeddings = semantic_model.encode(list(label_mapping.keys()))

def semantic_search(query, top_k=5):
    # Encode the query
    query_embedding = semantic_model.encode([query])
    
    # Calculate cosine similarity
    similarities = cosine_similarity(query_embedding, response_embeddings)[0]
    
    # Get top-k similar responses
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    return [(list(label_mapping.keys())[i], similarities[i]) for i in top_indices]
# --- semantic search --- #


# Function to map text labels to numeric labels
def map_labels(example):
    example['label'] = label_mapping[example['output']]
    return example

# Apply the label mapping
train_dataset = train_dataset.map(map_labels)
val_dataset = val_dataset.map(map_labels)
test_dataset = test_dataset.map(map_labels)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [14]:
# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [15]:
# Text cleaning function
def clean_text(text):
    # convert to lowercase
    text = text.lower()
    # remove special characters and digits
    text = re.sub(r'[^a-z\s]', '', text)
    # remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [16]:
# Tokenization function
def tokenize_function(examples):

    # clean the instruction and input text
    cleaned_instructions = [clean_text(inst) for inst in examples['instruction']]
    cleaned_inputs = [clean_text(inp) for inp in examples['input']]

    # combine cleaned instruction and input
    inputs = [f"{inst} {inp}" for inst, inp in zip(cleaned_instructions, cleaned_inputs)]

    tokenized_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs


# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Format for PyTorch
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [17]:
# Define model
num_labels = len(label_mapping)
model = BertForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=num_labels)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already be

In [19]:
# Define a custom Trainer to calculate loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        if labels is None:
            raise ValueError("Labels should not be None")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = F.cross_entropy(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Initialize Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

In [20]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,9.611000,9.649005
2,9.603100,10.275737
3,9.471700,11.732914


TrainOutput(global_step=4500, training_loss=9.561693901909722, metrics={'train_runtime': 13346.2937, 'train_samples_per_second': 2.697, 'train_steps_per_second': 0.337, 'total_flos': 1.0729306939392e+16, 'train_loss': 9.561693901909722, 'epoch': 3.0})

In [35]:
# Save the model
model_save_path = "./saved_model"
os.makedirs(model_save_path, exist_ok=True)
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to ./saved_model


In [25]:
# evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 9.649004936218262, 'eval_runtime': 70.4392, 'eval_samples_per_second': 21.295, 'eval_steps_per_second': 2.669, 'epoch': 3.0}


In [28]:
# Function to generate response category
def predict_category(instruction, input_text):
    # Clean the instruction and input text
    cleaned_instruction = clean_text(instruction)
    cleaned_input = clean_text(input_text)

    # Get BERT prediction
    inputs = tokenizer(f"{cleaned_instruction} {cleaned_input}", return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    bert_prediction = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_class_id)]

    # Get semantic search results
    semantic_results = semantic_search(f"{cleaned_instruction} {cleaned_input}")

    # If BERT prediction is in top semantic results, return it. otherwise, return top semantic result
    if bert_prediction in [result[0] for result in semantic_results]:
        return bert_prediction
    else:
        return semantic_results[0][0]  # Return the top semantic search result


# Test the model with a sample input
sample_instruction = "If you are a doctor, please answer the medical questions based on the patient's description."
sample_input = "I have a 13 month old baby who is very congested with a terrible cough. Its rattly/raspy and croupy sounding cough. She started choking on her coughs and the mucous that has come up. She also has a fever and runny nose. Should i take her to urgent care?"
predicted_label = predict_category(sample_instruction, sample_input)
print(f"Instruction: {sample_instruction}")
print(f"Input: {sample_input}")
print(f"Predicted Response: {predicted_label}")

Instruction: If you are a doctor, please answer the medical questions based on the patient's description.
Input: I have a 13 month old baby who is very congested with a terrible cough. Its rattly/raspy and croupy sounding cough. She started choking on her coughs and the mucous that has come up. She also has a fever and runny nose. Should i take her to urgent care?
Predicted Response: Hi, fever, cough and cold in a 13-month-old child suggest upper respiratory tract infection. Since you are already giving antipyretic medicine for 5 days, the child should be given antibiotic depending on the chest examination. Kindly get the child examined by doctor so that antibiotic can be started. Blood test like complete blood count, CRP level should be done to know the causes of fever. Take care.


In [34]:
# more evaluation

